# Computer Vision Final Exam Summary

## Comprehensive Lab Works Summary

This notebook consolidates all lab works from:
- **CNN** (Convolutional Neural Networks)
- **OD** (Object Detection - YOLO)
- **OT** (Object Tracking)
- **IS** (Image Segmentation)
- **GenAI** (Generative Models - GAN & VAE)
- **3D Vision** (PyTorch3D, Open3D, PointNet)

---

## Table of Contents

1. [CNN - Convolutional Neural Networks](#cnn)
2. [Object Detection - YOLO](#yolo)
3. [Object Tracking](#tracking)
4. [Image Segmentation](#segmentation)
5. [Generative Models](#generative)
6. [3D Vision](#3dvision)



# 1. CNN - Convolutional Neural Networks {#cnn}

## Key Concepts

### 1.1 Linear Classifiers

**Components:**
- **Weights and Bias Initialization**: Random small values for weights, zero for bias
- **Matrix Multiplication**: $y = X\theta + b$ or with augmented matrix $y = X_{bias} \theta_{bias}$
- **Activation Functions**:
  - **Sigmoid** (Binary): $\sigma(z) = \frac{1}{1 + e^{-z}}$
    - Output range: (0, 1)
    - Used for binary classification
  - **Softmax** (Multi-class): $\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$
    - Output: Probability distribution over K classes
    - Sum of outputs = 1
- **Loss Functions**:
  - **Binary Cross-Entropy**: $J(\theta) = -\frac{1}{m} \sum_{i=1}^{m} [y_i \log(\hat{y}_i) + (1-y_i) \log(1-\hat{y}_i)]$
  - **Categorical Cross-Entropy**: $J(\theta) = -\frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} y_{ik} \log(\hat{y}_{ik})$
- **Gradient Descent**: $\theta = \theta - \alpha \frac{\partial J(\theta)}{\partial \theta}$
  - **Batch GD**: Uses all samples
  - **Stochastic GD**: Uses one sample at a time
  - **Mini-batch GD**: Uses small batches

### 1.2 Multi-Layer Perceptron (MLP)

**Architecture:**
- Input Layer → Hidden Layer(s) with ReLU → Output Layer with Softmax
- **ReLU**: $f(x) = \max(0, x)$
  - Advantages: Non-saturating, sparse activations
  - Problem: Dead neurons (gradient = 0 for x < 0)
- **Backpropagation**: Chain rule for gradient computation through layers
- **Vanishing Gradient**: Problem in deep networks (solved by ReLU, BatchNorm)

### 1.3 Convolutional Neural Networks (CNN)

**Key Components:**
- **Convolutional Layers**: Extract spatial features using filters
  - **Filter/Kernel**: Small matrix that slides over input
  - **Feature Maps**: Output of convolution operation
  - **Stride**: Step size of filter movement
  - **Padding**: Add zeros around input to preserve size
- **Pooling Layers**: Reduce spatial dimensions
  - **MaxPool**: Takes maximum value in window
  - **AvgPool**: Takes average value in window
  - **Benefits**: Reduces parameters, provides translation invariance
- **Fully Connected Layers**: Final classification
- **Activation Functions**: ReLU, Softmax

**Output Size Formula:**
$$\text{Output Size} = \frac{W - K + 2P}{S} + 1$$
where:
- $W$ = input width/height
- $K$ = kernel size
- $P$ = padding
- $S$ = stride

**Example Calculation:**
- Input: 28×28, Kernel: 3×3, Padding: 0, Stride: 1
- Output: $\frac{28 - 3 + 2(0)}{1} + 1 = 26$
- After MaxPool(2): $\frac{26}{2} = 13$



In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")



## CNN Implementation Example

### Simple CNN Architecture
```
Input (1, 28, 28) 
→ Conv2d(1, 16, kernel_size=3) + ReLU 
→ MaxPool2d(2) 
→ Flatten 
→ Linear(13*13*16, 10) 
→ Output (10 classes)
```



In [ ]:
# Simple CNN Model
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        # Conv layer: (28-3+2*0)/1 + 1 = 26
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        # MaxPool: 26/2 = 13
        self.maxpool = nn.MaxPool2d(2)
        # FC layer: 13*13*16 = 2704
        self.fc1 = nn.Linear(13*13*16, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x, F.log_softmax(x, dim=1)

# Model initialization
cnn_model = MyCNN().to(device)
print(cnn_model)



## Transfer Learning with ResNet

**Key Points:**
- Use pre-trained models (e.g., ResNet-18) on ImageNet
- Modify final layer for your number of classes
- Options:
  - **Fine-tuning**: Update all weights
  - **Feature Extraction**: Freeze pre-trained layers, train only final layer



In [ ]:
# ResNet-18 Transfer Learning Example
from torchvision.models import ResNet18_Weights

# Load pre-trained ResNet-18
resnet18 = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Modify final layer for 10 classes (e.g., CIFAR-10)
num_classes = 10
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)

# For feature extraction (freeze pre-trained layers):
# for param in resnet18.parameters():
#     param.requires_grad = False
# resnet18.fc.requires_grad = True  # Only train final layer

resnet18 = resnet18.to(device)
print(f"ResNet-18 modified for {num_classes} classes")



---

# 2. Object Detection - YOLO {#yolo}

## YOLO (You Only Look Once) Architecture

### Key Concepts

**YOLO v1 Architecture:**
- **Input**: 448×448 image
- **Output**: 7×7 grid, each cell predicts:
  - 2 bounding boxes (B=2)
  - Confidence scores
  - 20 class probabilities (C=20 for VOC)
  - **Total output**: $S \times S \times (C + B \times 5) = 7 \times 7 \times 30 = 1470$

**Bounding Box Format:**
- $(x, y, w, h)$: center coordinates and dimensions (normalized 0-1)
- Confidence: $P(\text{object}) \times \text{IoU}$

**Key Features:**
- **Single Forward Pass**: Detects objects in one pass (faster than R-CNN)
- **Grid-based**: Divides image into S×S grid
- **Multi-scale**: Can detect objects of different sizes

### Loss Function Components

**Total Loss = Coordinate Loss + Confidence Loss + Class Loss**

1. **Box Coordinate Loss**: 
   $$\lambda_{coord} \sum_{i=0}^{S^2} \sum_{j=0}^{B} \mathbb{1}_{ij}^{obj} [(x_i - \hat{x}_i)^2 + (y_i - \hat{y}_i)^2]$$
   
2. **Box Size Loss** (with square root):
   $$\lambda_{coord} \sum_{i=0}^{S^2} \sum_{j=0}^{B} \mathbb{1}_{ij}^{obj} [(\sqrt{w_i} - \sqrt{\hat{w}_i})^2 + (\sqrt{h_i} - \sqrt{\hat{h}_i})^2]$$
   *Note: Square root makes small boxes and large boxes contribute equally*
   
3. **Object Confidence Loss**:
   $$\sum_{i=0}^{S^2} \sum_{j=0}^{B} \mathbb{1}_{ij}^{obj} (C_i - \hat{C}_i)^2$$
   
4. **No Object Confidence Loss**:
   $$\lambda_{noobj} \sum_{i=0}^{S^2} \sum_{j=0}^{B} \mathbb{1}_{ij}^{noobj} (C_i - \hat{C}_i)^2$$
   *Note: $\lambda_{noobj} = 0.5$ to reduce penalty for background*
   
5. **Class Loss**:
   $$\sum_{i=0}^{S^2} \mathbb{1}_{i}^{obj} \sum_{c \in classes} (p_i(c) - \hat{p}_i(c))^2$$

**Loss Weights:**
- $\lambda_{coord} = 5$ (emphasize coordinate accuracy)
- $\lambda_{noobj} = 0.5$ (reduce background penalty)

### Post-Processing

**Non-Maximum Suppression (NMS):**
1. Sort boxes by confidence score
2. Select box with highest confidence
3. Remove all boxes with IoU > threshold
4. Repeat until no boxes remain

**Key Metrics:**
- **IoU (Intersection over Union)**: $\text{IoU} = \frac{\text{Area of Intersection}}{\text{Area of Union}}$
- **mAP (mean Average Precision)**: Average precision across all classes
- **Precision**: $\frac{TP}{TP + FP}$
- **Recall**: $\frac{TP}{TP + FN}$



In [ ]:
# YOLO Architecture Config
architecture_config = [
    (7, 64, 2, 3),      # (kernel_size, filters, stride, padding)
    "M",                 # MaxPool
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],  # Repeated 4 times
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],  # Repeated 2 times
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1),
]

# CNN Block
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(CNNBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.leakyrelu = nn.LeakyReLU(0.1)
        
    def forward(self, x):
        return self.leakyrelu(self.batchnorm(self.conv(x)))

# Complete YOLO Model
class Yolov1(nn.Module):
    def __init__(self, in_channels=3, split_size=7, num_boxes=2, num_classes=20):
        super(Yolov1, self).__init__()
        self.architecture = architecture_config
        self.in_channels = in_channels
        self.darknet = self._create_conv_layers(self.architecture)
        self.fcs = self._create_fc(split_size, num_boxes, num_classes)
    
    def forward(self, x):
        x = self.darknet(x)
        return self.fcs(torch.flatten(x, start_dim=1))
    
    def _create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels
        
        for x in architecture:
            if type(x) == tuple:
                layers += [CNNBlock(in_channels, out_channels=x[1], 
                                  kernel_size=x[0], stride=x[2], padding=x[3])]
                in_channels = x[1]
            elif type(x) == str:
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            elif type(x) == list:
                conv1 = x[0]
                conv2 = x[1]
                num_repeats = x[2]
                for _ in range(num_repeats):
                    layers += [CNNBlock(in_channels, out_channels=conv1[1],
                                      kernel_size=conv1[0], stride=conv1[2], padding=conv1[3])]
                    layers += [CNNBlock(conv1[1], out_channels=conv2[1],
                                      kernel_size=conv2[0], stride=conv2[2], padding=conv2[3])]
                    in_channels = conv2[1]
        return nn.Sequential(*layers)
    
    def _create_fc(self, split_size, num_boxes, num_classes):
        S, B, C = split_size, num_boxes, num_classes
        return nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024*S*S, 4096),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.1),
            nn.Linear(4096, S*S*(C + B*5)),
        )

# IoU Calculation
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    """
    Calculate IoU between predicted and ground truth boxes
    """
    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2
    elif box_format == "corners":
        box1_x1, box1_y1 = boxes_preds[..., 0:1], boxes_preds[..., 1:2]
        box1_x2, box1_y2 = boxes_preds[..., 2:3], boxes_preds[..., 3:4]
        box2_x1, box2_y1 = boxes_labels[..., 0:1], boxes_labels[..., 1:2]
        box2_x2, box2_y2 = boxes_labels[..., 2:3], boxes_labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))
    
    return intersection / (box1_area + box2_area - intersection + 1e-6)

# Non-Maximum Suppression
def non_max_suppression(bboxes, iou_threshold, threshold, box_format="corners"):
    """
    Apply NMS to remove overlapping bounding boxes
    """
    bboxes = [box for box in bboxes if box[1] > threshold]
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    bboxes_after_nms = []
    
    while bboxes:
        chosen_box = bboxes.pop(0)
        bboxes = [
            box for box in bboxes
            if box[0] != chosen_box[0]
            or intersection_over_union(
                torch.tensor(chosen_box[2:]),
                torch.tensor(box[2:]),
                box_format=box_format,
            ) < iou_threshold
        ]
        bboxes_after_nms.append(chosen_box)
    
    return bboxes_after_nms

print("YOLO components defined")



In [ ]:
# Object Tracking Examples

# 1. Color-Based Tracking
from collections import deque
import cv2
import numpy as np

def color_based_tracking(frame, color_lower, color_upper, buffer_size=64):
    """
    Track object by color in HSV space
    """
    # Convert to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Create mask
    mask = cv2.inRange(hsv, color_lower, color_upper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    # Find contours
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    center = None
    if len(cnts) > 0:
        # Find largest contour
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        if M["m00"] != 0:
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    
    return center, mask

# 2. Single Object Tracking
def single_object_tracking_example():
    """
    Single object tracking using OpenCV trackers
    """
    tracker_types = ['BOOSTING', 'MIL', 'KCF', 'GOTURN']
    tracker_type = 'KCF'
    
    # Create tracker
    if tracker_type == 'BOOSTING':
        tracker = cv2.legacy.TrackerBoosting_create()
    elif tracker_type == 'MIL':
        tracker = cv2.legacy.TrackerMIL_create()
    elif tracker_type == 'KCF':
        tracker = cv2.legacy.TrackerKCF_create()
    elif tracker_type == 'GOTURN':
        tracker = cv2.legacy.TrackerGOTURN_create()
    
    video = cv2.VideoCapture(0)
    ok, frame = video.read()
    if not ok:
        return
    
    bbox = cv2.selectROI(frame, False)
    ok = tracker.init(frame, bbox)
    
    while True:
        ok, frame = video.read()
        if not ok:
            break
        
        ok, bbox = tracker.update(frame)
        
        if ok:
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        else:
            cv2.putText(frame, "Tracking failure", (100, 80),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        
        cv2.imshow("Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    video.release()
    cv2.destroyAllWindows()

# 3. Multi-Object Tracking
def multi_object_tracking_example():
    """
    Multi-object tracking using MultiTracker
    """
    tracker_type = "KCF"
    
    if hasattr(cv2, "legacy"):
        FACTORIES = {
            "KCF": cv2.legacy.TrackerKCF_create,
            "MIL": cv2.legacy.TrackerMIL_create,
        }
        trackers = cv2.legacy.MultiTracker_create()
    else:
        FACTORIES = {
            "KCF": getattr(cv2, "TrackerKCF_create", None),
            "MIL": getattr(cv2, "TrackerMIL_create", None),
        }
        trackers = cv2.MultiTracker_create()
    
    video = cv2.VideoCapture('video.mp4')
    
    while True:
        ok, frame = video.read()
        if not ok:
            break
        
        frame = cv2.resize(frame, (720, 640))
        success, boxes = trackers.update(frame)
        
        if success:
            for b in boxes:
                x, y, w, h = map(int, b)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        else:
            cv2.putText(frame, "Tracking failure", (100, 80),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord("s"):
            # Add new tracker
            box = cv2.selectROI("Frame", frame, fromCenter=False, showCrosshair=True)
            tr = FACTORIES[tracker_type]()
            trackers.add(tr, frame, box)
        elif key == ord("q"):
            break
    
    video.release()
    cv2.destroyAllWindows()

# 4. Non-Maximum Suppression (NMS) for Tracking
def nms_boxes(boxes, iou_thr=0.3):
    """
    Apply Non-Maximum Suppression to remove overlapping boxes
    """
    if not boxes:
        return []
    
    b = np.array(boxes, dtype=np.float32)
    x1, y1 = b[:, 0], b[:, 1]
    x2, y2 = b[:, 0] + b[:, 2], b[:, 1] + b[:, 3]
    areas = (x2 - x1) * (y2 - y1)
    order = areas.argsort()[::-1]
    
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        iw = np.maximum(0.0, xx2 - xx1)
        ih = np.maximum(0.0, yy2 - yy1)
        inter = iw * ih
        iou = inter / (areas[i] + areas[order[1:]] - inter + 1e-6)
        order = order[1:][iou < iou_thr]
    
    return [tuple(map(int, b[i])) for i in keep]

print("Tracking functions defined")



---

# 4. Image Segmentation {#segmentation}

## Segmentation Methods

### 4.1 Classical Methods

#### Mean Shift Clustering
- **Non-parametric clustering algorithm**
- Groups pixels by iteratively shifting towards higher density
- Good for color-based segmentation
- **Bandwidth**: Controls cluster size (smaller = more clusters)

**Algorithm:**
1. Estimate bandwidth from data
2. For each point, shift towards mean of points in bandwidth window
3. Iterate until convergence
4. Merge nearby modes

#### K-Means Clustering
- **Partition pixels into K clusters**
- Minimize within-cluster variance
- Fast and simple
- **K**: Number of clusters (must be specified)

**Algorithm:**
1. Initialize K cluster centers (random or K-means++)
2. Assign each pixel to nearest center
3. Update centers as mean of assigned pixels
4. Repeat until convergence or max iterations

**Criteria:**
- **TERM_CRITERIA_EPS**: Stop when change < epsilon
- **TERM_CRITERIA_MAX_ITER**: Stop after max iterations
- **KMEANS_PP_CENTERS**: K-means++ initialization

### 4.2 Deep Learning Methods

#### U-Net Architecture

**Key Features:**
- **Encoder (Contracting Path)**: Downsampling, feature extraction
  - Uses VGG16 or similar backbone
  - Progressively reduces spatial size, increases channels
- **Bottleneck**: Deepest layer with highest-level features
  - Smallest spatial size, most channels
- **Decoder (Expansive Path)**: Upsampling, spatial recovery
  - Progressively increases spatial size, decreases channels
  - Uses transposed convolution (deconvolution)
- **Skip Connections**: Preserve fine-grained details
  - Concatenate encoder features with decoder at same level
  - Helps recover spatial information lost in downsampling

**Architecture:**
```
Input (256×256) 
→ Encoder Block 1 (128×128) ────┐
→ Encoder Block 2 (64×64) ──────┤
→ Encoder Block 3 (32×32) ──────┤
→ Encoder Block 4 (16×16) ──────┤
→ Encoder Block 5 (8×8) ────────┤
→ Bottleneck (8×8)               │
→ Decoder Block 1 (16×16) ←─────┘ (skip from Block 4)
→ Decoder Block 2 (32×32) ←─────── (skip from Block 3)
→ Decoder Block 3 (64×64) ←─────── (skip from Block 2)
→ Decoder Block 4 (128×128) ←───── (skip from Block 1)
→ Output (256×256)
```

**Loss Function:**
- **Cross-Entropy Loss**: For pixel-wise classification
  - $L_{CE} = -\sum_{i} y_i \log(\hat{y}_i)$
- **Dice Loss**: For imbalanced classes
  - $L_{Dice} = 1 - \frac{2|X \cap Y|}{|X| + |Y|}$
- **Combined Loss**: $L = L_{CE} + \lambda L_{Dice}$

**Key Components:**
- **Conv Block**: Conv2d → BatchNorm → ReLU
- **Up Conv**: Transposed Convolution (Deconvolution)
  - Increases spatial dimensions
  - Can have checkerboard artifacts
- **Skip Connections**: Concatenate encoder features with decoder
  - Channel dimension doubles: `torch.cat([decoder_feat, encoder_feat], dim=1)`



In [ ]:
# Classical Segmentation Methods

# 1. K-Means Segmentation
import cv2
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.preprocessing import MinMaxScaler

def kmeans_segmentation(image, K=4):
    """
    K-Means segmentation example
    """
    # Reshape image to 2D array of pixels
    img2D = image.reshape((-1, 3))
    img2D = np.float32(img2D)
    
    # Define criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    
    # Apply K-Means
    ret, label, center = cv2.kmeans(img2D, K, None, criteria, 10, cv2.KMEANS_PP_CENTERS)
    
    # Convert back to uint8
    center = np.uint8(center)
    res = center[label.flatten()]
    result = res.reshape(image.shape)
    
    return result, label, center

# 2. Mean Shift Segmentation
def mean_shift_segmentation(image, quantile=0.04):
    """
    Mean Shift clustering for segmentation
    """
    # Resize and reshape
    img = cv2.resize(image, None, fx=0.5, fy=0.5)
    img2D = img.reshape((-1, 3))
    img2D = np.float32(img2D)
    
    # Normalize
    norm_img2d = MinMaxScaler(feature_range=(0, 1)).fit_transform(img2D)
    
    # Estimate bandwidth
    bandwidth = estimate_bandwidth(norm_img2d, quantile=quantile, n_jobs=2)
    
    # Apply Mean Shift
    ms_res = MeanShift(bandwidth=bandwidth, n_jobs=2, bin_seeding=True, cluster_all=True).fit(norm_img2d)
    
    # Get labels and reshape
    labels = ms_res.labels_
    segmented_image = labels.reshape(img.shape[:2])
    
    return segmented_image, ms_res.cluster_centers_

# U-Net Complete Implementation
def conv_block(in_channels, out_channels):
    """Convolutional block with BatchNorm and ReLU"""
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

def up_conv_block(in_channels, out_channels):
    """Up-convolution block"""
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
        nn.ReLU(inplace=True)
    )

# Complete U-Net Model
class UNet(nn.Module):
    def __init__(self, pretrained=True, out_channels=12):
        super().__init__()
        # Use VGG16 as encoder
        self.encoder = models.vgg16_bn(pretrained=pretrained).features
        self.block1 = nn.Sequential(*self.encoder[:6])
        self.block2 = nn.Sequential(*self.encoder[6:13])
        self.block3 = nn.Sequential(*self.encoder[13:20])
        self.block4 = nn.Sequential(*self.encoder[20:27])
        self.block5 = nn.Sequential(*self.encoder[27:34])
        
        self.bottleneck = nn.Sequential(*self.encoder[34:])
        self.conv_bottleneck = conv_block(512, 1024)
        
        # Decoder with skip connections
        self.up_conv6 = up_conv_block(1024, 512)
        self.conv6 = conv_block(512 + 512, 512)
        self.up_conv7 = up_conv_block(512, 256)
        self.conv7 = conv_block(256 + 512, 256)
        self.up_conv8 = up_conv_block(256, 128)
        self.conv8 = conv_block(128 + 256, 128)
        self.up_conv9 = up_conv_block(128, 64)
        self.conv9 = conv_block(64 + 128, 64)
        self.up_conv10 = up_conv_block(64, 32)
        self.conv10 = conv_block(32 + 64, 32)
        self.conv11 = nn.Conv2d(32, out_channels, kernel_size=1)
    
    def forward(self, x):
        # Encoder
        block1 = self.block1(x)
        block2 = self.block2(block1)
        block3 = self.block3(block2)
        block4 = self.block4(block3)
        block5 = self.block5(block4)
        
        bottleneck = self.bottleneck(block5)
        x = self.conv_bottleneck(bottleneck)
        
        # Decoder with skip connections
        x = self.up_conv6(x)
        x = torch.cat([x, block5], dim=1)
        x = self.conv6(x)
        
        x = self.up_conv7(x)
        x = torch.cat([x, block4], dim=1)
        x = self.conv7(x)
        
        x = self.up_conv8(x)
        x = torch.cat([x, block3], dim=1)
        x = self.conv8(x)
        
        x = self.up_conv9(x)
        x = torch.cat([x, block2], dim=1)
        x = self.conv9(x)
        
        x = self.up_conv10(x)
        x = torch.cat([x, block1], dim=1)
        x = self.conv10(x)
        
        x = self.conv11(x)
        return x

# U-Net Loss Function
def unet_loss(preds, targets):
    """
    U-Net loss: Cross-entropy for pixel-wise classification
    """
    targets = targets.squeeze(1).long()
    ce_loss = nn.CrossEntropyLoss()(preds, targets)
    acc = (torch.max(preds, 1)[1] == targets).float().mean()
    return ce_loss, acc

print("Segmentation functions defined")



---

# 5. Generative Models {#generative}

## 5.1 Generative Adversarial Networks (GAN)

### Architecture

**Two Networks:**
1. **Generator (G)**: Creates fake images from noise
   - Input: Random noise vector $z \sim \mathcal{N}(0, 1)$ or $\mathcal{U}(-1, 1)$
   - Output: Generated image $G(z)$
   - Goal: Fool discriminator into thinking generated images are real
   
2. **Discriminator (D)**: Distinguishes real from fake
   - Input: Real or generated image
   - Output: Probability of being real $D(x) \in [0, 1]$
   - Goal: Correctly classify real vs fake

### Training Process

**Minimax Game:**
$$\min_G \max_D V(D, G) = \mathbb{E}_{x \sim p_{data}(x)}[\log D(x)] + \mathbb{E}_{z \sim p_z(z)}[\log(1 - D(G(z)))]$$

**Training Steps:**
1. **Train Discriminator** (k steps, usually k=1):
   - Real images: maximize $\log D(x)$ → minimize $-\log D(x)$
   - Fake images: maximize $\log(1 - D(G(z)))$ → minimize $-\log(1 - D(G(z)))$
   - Combined: $L_D = -\frac{1}{2}[\log D(x) + \log(1 - D(G(z)))]$
   
2. **Train Generator** (1 step):
   - **Saturating**: Minimize $\log(1 - D(G(z)))$ (can have vanishing gradients)
   - **Non-saturating**: Maximize $\log D(G(z))$ → minimize $-\log D(G(z))$ (better)
   - Use flipped labels: treat fake as real

**Loss Functions:**
- **Discriminator Loss**: $L_D = -\frac{1}{2}[\log D(x) + \log(1 - D(G(z)))]$
- **Generator Loss**: $L_G = -\log D(G(z))$ (non-saturating)

**Training Tips:**
- Train D and G alternately
- Use same learning rate for both (typically 0.0002)
- Use Adam optimizer with betas=(0.5, 0.9)
- Label smoothing: Use 0.9 instead of 1.0 for real labels

### Key Components

**Generator:**
- Fully connected or convolutional layers
- Batch normalization (except input layer)
- ReLU/LeakyReLU activations
- Tanh output (normalized to [-1, 1])
- No dropout in generator

**Discriminator:**
- Binary classifier
- Sigmoid output
- Dropout for regularization (p=0.3-0.5)
- LeakyReLU (negative slope=0.2)

## 5.2 Variational Autoencoders (VAE)

### Architecture

**Encoder-Decoder Structure:**
- **Encoder**: Maps input $x$ to latent distribution parameters $(\mu, \sigma^2)$
  - Outputs: $\mu$ (mean) and $\log(\sigma^2)$ (log variance)
  - Why log? Ensures $\sigma^2 > 0$ (always positive)
- **Reparameterization Trick**: Sample $z = \mu + \sigma \cdot \epsilon$ where $\epsilon \sim \mathcal{N}(0, 1)$
  - **Why needed?**: Allows backpropagation through random sampling
  - Without it: sampling is non-differentiable
  - With it: $\frac{\partial z}{\partial \mu} = 1$, $\frac{\partial z}{\partial \sigma} = \epsilon$
- **Decoder**: Reconstructs $x$ from latent $z$
  - Output: Reconstructed image $\hat{x}$

### Loss Function

**VAE Loss = Reconstruction Loss + KL Divergence**

$$L_{VAE} = \mathbb{E}_{q_\phi(z|x)}[\log p_\theta(x|z)] - D_{KL}(q_\phi(z|x) || p(z))$$

**Components:**
1. **Reconstruction Loss**: 
   - MSE: $||x - \hat{x}||^2$ (for continuous data)
   - BCE: $-\sum [x \log(\hat{x}) + (1-x)\log(1-\hat{x})]$ (for binary data)
   - Measures how well decoder reconstructs input
   
2. **KL Divergence**: Regularizes latent space to standard normal $\mathcal{N}(0, I)$
   - Ensures latent space is well-structured
   - Allows smooth interpolation
   - Formula: $D_{KL} = -\frac{1}{2} \sum_{i=1}^{d} [1 + \log(\sigma_i^2) - \mu_i^2 - \sigma_i^2]$

**Interpretation:**
- **Reconstruction Term**: Maximize likelihood of data
- **KL Term**: Minimize distance from prior (standard normal)
- **Trade-off**: More KL weight → better structure, worse reconstruction

### Key Differences: GAN vs VAE

| Aspect | GAN | VAE |
|--------|-----|-----|
| Training | Adversarial (minimax game) | Reconstruction + Regularization |
| Latent Space | No explicit structure | Structured (Gaussian) |
| Generation Quality | High quality, sharp | Blurrier, smoother |
| Interpolation | Not guaranteed smooth | Smooth in latent space |
| Mode Collapse | Can occur | Less common |
| Training Stability | Can be unstable | More stable |
| Use Case | High-quality generation | Latent space exploration |



In [ ]:
# GAN Generator Example
class Generator(nn.Module):
    def __init__(self, latent_dim=100, img_dim=(1, 28, 28)):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.img_dim = img_dim
        
        def block(in_feat, out_feat, normalize=True, dropout=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            if dropout:
                layers.append(nn.Dropout(p=0.5))
            return layers
        
        self.model = nn.Sequential(
            *block(self.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            nn.Linear(512, int(np.prod(img_dim))),
            nn.Tanh()
        )
    
    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *self.img_dim)
        return img

# GAN Discriminator Example
class Discriminator(nn.Module):
    def __init__(self, img_dim=(1, 28, 28)):
        super(Discriminator, self).__init__()
        self.img_dim = img_dim
        
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(self.img_dim)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# VAE Example
class VAE(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, stride=2, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),
            nn.Conv2d(32, 64, stride=2, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            nn.Flatten(),
        )
        
        self.z_mean = nn.Linear(3136, 2)  # 7x7x64 = 3136
        self.z_log_var = nn.Linear(3136, 2)
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(2, 3136),
            nn.Unflatten(1, (64, 7, 7)),
            nn.ConvTranspose2d(64, 64, stride=2, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            nn.ConvTranspose2d(64, 32, stride=2, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),
            nn.ConvTranspose2d(32, 1, stride=2, kernel_size=3, padding=1),
            nn.Sigmoid()
        )
    
    def reparameterize(self, z_mu, z_log_var):
        """Reparameterization trick"""
        eps = torch.randn(z_mu.size(0), z_mu.size(1)).to(z_mu.device)
        z = z_mu + eps * torch.exp(z_log_var / 2.)
        return z
    
    def forward(self, x):
        x = self.encoder(x)
        z_mean, z_log_var = self.z_mean(x), self.z_log_var(x)
        encoded = self.reparameterize(z_mean, z_log_var)
        decoded = self.decoder(encoded)
        return encoded, z_mean, z_log_var, decoded

print("Generative models defined")



In [ ]:
# PyTorch3D Mesh Rendering
try:
    import pytorch3d
    import pytorch3d.renderer
    import pytorch3d.structures
    import pytorch3d.io
    
    def setup_mesh_renderer(device='cuda'):
        """Setup mesh renderer with camera and lights"""
        # Camera setup
        R = torch.eye(3).unsqueeze(0)  # Identity rotation
        T = torch.tensor([[0, 0, 3]])  # Camera at z=3
        
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
            R=R, T=T, fov=60, device=device
        )
        
        # Renderer setup
        image_size = 512
        raster_settings = pytorch3d.renderer.RasterizationSettings(
            image_size=image_size
        )
        rasterizer = pytorch3d.renderer.MeshRasterizer(
            raster_settings=raster_settings
        )
        shader = pytorch3d.renderer.HardPhongShader(device=device)
        renderer = pytorch3d.renderer.MeshRenderer(
            rasterizer=rasterizer, shader=shader
        )
        
        return renderer, cameras
    
    def load_and_render_mesh(obj_path, device='cuda'):
        """Load mesh and render it"""
        # Load mesh
        vertices, face_props, text_props = pytorch3d.io.load_obj(obj_path)
        faces = face_props.verts_idx
        
        # Batch the tensors
        vertices = vertices.unsqueeze(0).to(device)
        faces = faces.unsqueeze(0).to(device)
        
        # Create texture
        texture_rgb = torch.ones_like(vertices) * torch.tensor([0.7, 0.7, 1]).to(device)
        textures = pytorch3d.renderer.TexturesVertex(texture_rgb)
        
        # Create mesh
        meshes = pytorch3d.structures.Meshes(
            verts=vertices, faces=faces, textures=textures
        )
        
        # Setup renderer
        renderer, cameras = setup_mesh_renderer(device)
        
        # Render
        image = renderer(meshes, cameras=cameras)
        return image[0].cpu().numpy()
    
    print("PyTorch3D functions defined")
except ImportError:
    print("PyTorch3D not available")

# Open3D Examples
try:
    import open3d as o3d
    
    def open3d_mesh_operations():
        """Open3D mesh processing examples"""
        # Load mesh
        mesh = o3d.io.read_triangle_mesh("mesh.obj")
        
        # Compute normals
        mesh.compute_vertex_normals()
        
        # Mesh filtering
        mesh_smooth = mesh.filter_smooth_simple(number_of_iterations=5)
        mesh_laplacian = mesh.filter_smooth_laplacian(number_of_iterations=10)
        
        # Point sampling
        pcd = mesh.sample_points_uniformly(number_of_points=500)
        
        # Mesh properties
        is_watertight = mesh.is_watertight()
        is_orientable = mesh.is_orientable()
        
        return mesh, pcd
    
    print("Open3D functions defined")
except ImportError:
    print("Open3D not available")

# Complete PointNet Implementation
class Tnet(nn.Module):
    """Transform network for PointNet"""
    def __init__(self, k=3):
        super().__init__()
        self.k = k
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k*k)
        
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)
    
    def forward(self, input):
        bs = input.size(0)
        xb = F.relu(self.bn1(self.conv1(input)))
        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = F.relu(self.bn3(self.conv3(xb)))
        pool = nn.MaxPool1d(xb.size(-1))(xb)
        flat = nn.Flatten(1)(pool)
        xb = F.relu(self.bn4(self.fc1(flat)))
        xb = F.relu(self.bn5(self.fc2(xb)))
        
        # Initialize as identity
        init = torch.eye(self.k, requires_grad=True).repeat(bs, 1, 1)
        if xb.is_cuda:
            init = init.cuda()
        matrix = self.fc3(xb).view(-1, self.k, self.k) + init
        return matrix

class PointNet(nn.Module):
    """Complete PointNet for classification"""
    def __init__(self, num_classes=10):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)
    
    def forward(self, input):
        # Input transform
        matrix3x3 = self.input_transform(input)
        xb = torch.bmm(torch.transpose(input, 1, 2), matrix3x3).transpose(1, 2)
        
        xb = F.relu(self.bn1(self.conv1(xb)))
        
        # Feature transform
        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb, 1, 2), matrix64x64).transpose(1, 2)
        
        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        
        # Classification head
        xb = F.relu(self.bn4(self.fc1(output)))
        xb = F.relu(self.bn5(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        
        return self.logsoftmax(output), matrix3x3, matrix64x64

# PointNet Loss
def pointnet_loss(outputs, labels, m3x3, m64x64, alpha=0.0001):
    """PointNet loss with transform regularization"""
    criterion = nn.NLLLoss()
    bs = outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs, 1, 1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs, 1, 1)
    if outputs.is_cuda:
        id3x3 = id3x3.cuda()
        id64x64 = id64x64.cuda()
    diff3x3 = id3x3 - torch.bmm(m3x3, m3x3.transpose(1, 2))
    diff64x64 = id64x64 - torch.bmm(m64x64, m64x64.transpose(1, 2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3) + torch.norm(diff64x64)) / float(bs)

print("3D Vision components defined")



## Summary of Key Formulas

### CNN
- **Output Size**: $\frac{W - K + 2P}{S} + 1$
- **Parameters**: $\text{params} = (K \times K \times C_{in} + 1) \times C_{out}$
- **Pooling Output**: $\frac{W}{S}$ (for stride S)

### YOLO
- **Output Size**: $S \times S \times (C + B \times 5)$
- **IoU**: $\text{IoU} = \frac{\text{Intersection}}{\text{Union}} = \frac{\text{Area}(B_1 \cap B_2)}{\text{Area}(B_1 \cup B_2)}$
- **mAP**: Mean Average Precision across all classes
- **Confidence**: $P(\text{object}) \times \text{IoU}_{pred}^{truth}$

### GAN
- **Discriminator Loss**: $L_D = -\frac{1}{2}[\log D(x) + \log(1 - D(G(z)))]$
- **Generator Loss**: $L_G = -\log D(G(z))$ (non-saturating)
- **Minimax Objective**: $\min_G \max_D V(D, G)$

### VAE
- **Total Loss**: $L = \text{Reconstruction} + D_{KL}(q_\phi(z|x) || p(z))$
- **KL Divergence**: $D_{KL} = -\frac{1}{2} \sum_{i=1}^{d} [1 + \log(\sigma_i^2) - \mu_i^2 - \sigma_i^2]$
- **Reparameterization**: $z = \mu + \sigma \cdot \epsilon$ where $\epsilon \sim \mathcal{N}(0, 1)$

### PointNet
- **Loss**: $L = L_{cls} + \alpha(||I - A_3 A_3^T||_F^2 + ||I - A_{64} A_{64}^T||_F^2)$
- **Max Pooling**: Makes network permutation invariant

### Segmentation
- **Dice Coefficient**: $\text{Dice} = \frac{2|X \cap Y|}{|X| + |Y|}$
- **Pixel Accuracy**: $\frac{\text{Correct Pixels}}{\text{Total Pixels}}$

---

## Evaluation Metrics

### Classification
- **Accuracy**: $\frac{TP + TN}{TP + TN + FP + FN}$
- **Precision**: $\frac{TP}{TP + FP}$
- **Recall**: $\frac{TP}{TP + FN}$
- **F1-Score**: $\frac{2 \times \text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$

### Object Detection
- **IoU**: Intersection over Union
- **mAP**: Mean Average Precision
- **AP@0.5**: Average Precision at IoU threshold 0.5

### Segmentation
- **Pixel Accuracy**: Percentage of correctly classified pixels
- **Mean IoU**: Average IoU across all classes
- **Dice Coefficient**: Overlap measure

---

## Training Tips & Best Practices

### CNN Training
1. **Data Augmentation**: Rotation, flipping, scaling
2. **Batch Normalization**: Stabilizes training
3. **Learning Rate Scheduling**: Reduce on plateau
4. **Early Stopping**: Prevent overfitting

### YOLO Training
1. **Anchor Boxes**: Predefined box shapes
2. **Multi-scale Training**: Train on different image sizes
3. **Data Augmentation**: Mosaic, mixup
4. **Loss Balancing**: $\lambda_{coord} = 5$, $\lambda_{noobj} = 0.5$

### GAN Training
1. **Balance**: Train D and G alternately
2. **Learning Rates**: Usually same for both networks
3. **Batch Normalization**: Use in generator
4. **Label Smoothing**: Prevent discriminator from being too confident

### VAE Training
1. **KL Weight**: Balance reconstruction vs regularization
2. **Beta-VAE**: Weight KL term for better disentanglement
3. **Latent Dimension**: Choose based on data complexity

---

## Exam Tips

### Key Concepts to Master

1. **CNN**:
   - Understand convolution operation and how dimensions change
   - Know pooling effects (MaxPool, AvgPool)
   - Transfer learning strategies

2. **YOLO**:
   - Output format: $S \times S \times (C + B \times 5)$
   - Loss function components
   - IoU calculation and NMS

3. **Tracking**:
   - Different tracker types (KCF, MIL, etc.)
   - Detection + tracking pipeline
   - Multi-object tracking challenges

4. **Segmentation**:
   - U-Net architecture with skip connections
   - Classical vs deep learning methods
   - Loss functions for segmentation

5. **GAN/VAE**:
   - Training process (adversarial vs reconstruction)
   - Loss functions and their meanings
   - Reparameterization trick for VAE

6. **3D Vision**:
   - Mesh rendering pipeline
   - PointNet architecture and permutation invariance
   - Transform networks

### Practice Problems

1. **Calculate CNN output size**: Given input 224×224, kernel 3×3, padding 1, stride 1
2. **Calculate YOLO output**: Given S=7, C=20, B=2
3. **Compute IoU**: Given two bounding boxes
4. **Implement forward pass**: For any architecture
5. **Explain skip connections**: Why they're important in U-Net

### Common Mistakes to Avoid

1. **Dimension Mismatches**: Always check tensor shapes
2. **Loss Function**: Use correct loss for task (CE for classification, MSE for regression)
3. **Batch Dimension**: Remember to batch all tensors
4. **Device Mismatch**: Ensure all tensors on same device (CPU/GPU)
5. **Gradient Flow**: Check if requires_grad is set correctly

---

## Quick Reference

### PyTorch Essentials
```python
# Model definition
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(10, 1)
    
    def forward(self, x):
        return self.layer(x)

# Training loop
model.train()
for batch in dataloader:
    optimizer.zero_grad()
    output = model(input)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    output = model(input)
```

### Common Operations
- **Flatten**: `torch.flatten(x, start_dim=1)`
- **Concatenate**: `torch.cat([x1, x2], dim=1)`
- **Reshape**: `x.view(batch, channels, height, width)`
- **Transpose**: `x.transpose(1, 2)` or `x.permute(0, 2, 1)`

---

**Good luck with your exam!** 🎓



## Complete Training Loop Examples

### CNN Training Loop
```python
def train_cnn(model, train_loader, val_loader, epochs, device):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs, _ = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs, _ = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        train_losses.append(train_loss/len(train_loader))
        train_accs.append(100*train_correct/train_total)
        val_losses.append(val_loss/len(val_loader))
        val_accs.append(100*val_correct/val_total)
        
        print(f'Epoch [{epoch+1}/{epochs}]')
        print(f'Train Loss: {train_losses[-1]:.4f}, Acc: {train_accs[-1]:.2f}%')
        print(f'Val Loss: {val_losses[-1]:.4f}, Acc: {val_accs[-1]:.2f}%')
    
    return train_losses, train_accs, val_losses, val_accs
```

### GAN Training Loop
```python
def train_gan(generator, discriminator, train_loader, epochs, device):
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    adversarial_loss = nn.BCELoss()
    
    for epoch in range(epochs):
        for batch_idx, (real_images, _) in enumerate(train_loader):
            batch_size = real_images.size(0)
            real_images = real_images.to(device)
            real_labels = torch.ones(batch_size, device=device)
            fake_labels = torch.zeros(batch_size, device=device)
            
            # Train Discriminator
            optimizer_D.zero_grad()
            
            # Real images
            discr_pred_real = discriminator(real_images).view(-1)
            real_loss = adversarial_loss(discr_pred_real, real_labels)
            
            # Fake images
            z = torch.randn(batch_size, 100, device=device)
            fake_images = generator(z)
            discr_pred_fake = discriminator(fake_images.detach()).view(-1)
            fake_loss = adversarial_loss(discr_pred_fake, fake_labels)
            
            discr_loss = 0.5 * (real_loss + fake_loss)
            discr_loss.backward()
            optimizer_D.step()
            
            # Train Generator
            optimizer_G.zero_grad()
            discr_pred_fake = discriminator(fake_images).view(-1)
            gener_loss = adversarial_loss(discr_pred_fake, real_labels)  # Flip labels
            gener_loss.backward()
            optimizer_G.step()
            
            if batch_idx % 100 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Batch [{batch_idx}/{len(train_loader)}]')
                print(f'D Loss: {discr_loss.item():.4f}, G Loss: {gener_loss.item():.4f}')
```

### VAE Training Loop
```python
def train_vae(model, train_loader, epochs, device):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        recon_loss_total = 0.0
        kl_loss_total = 0.0
        
        for batch_idx, (images, _) in enumerate(train_loader):
            images = images.to(device)
            
            optimizer.zero_grad()
            encoded, z_mean, z_log_var, decoded = model(images)
            
            # KL Divergence
            kl_div = -0.5 * torch.sum(1 + z_log_var - z_mean**2 - torch.exp(z_log_var), axis=1)
            kl_div = kl_div.mean()
            
            # Reconstruction Loss (MSE)
            pixelwise = F.mse_loss(decoded, images, reduction='none')
            pixelwise = pixelwise.view(images.size(0), -1).sum(axis=1).mean()
            
            loss = pixelwise + kl_div
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            recon_loss_total += pixelwise.item()
            kl_loss_total += kl_div.item()
        
        print(f'Epoch [{epoch+1}/{epochs}]')
        print(f'Total Loss: {total_loss/len(train_loader):.4f}')
        print(f'Recon Loss: {recon_loss_total/len(train_loader):.4f}, KL Loss: {kl_loss_total/len(train_loader):.4f}')
```

